In [2]:
import json
import ipywidgets as widgets
import os
import time
import numpy as np
import random
import pandas as pd

In [89]:


# set up the helper routines to load the jsons
# written by J Wallin

def jsonFromFile(fname):
    f = open(fname,"r")
    s = f.read()
    f.close()
    return json.loads(s)

def jsonToFile(fname):
    print(fname)
    
def listFiles(assetPath, printFiles):
    finalList = []
    fullList = os.listdir(assetPath)
    for f in fullList:
        if f.find("json") == len(f)-4:
            print(f)
            finalList.append(f)
    return finalList

def loadFileIntoDictionary(path, flist):
    jdata = {}
    for f in flist:
        jdata[f] = jsonFromFile(path + f)
    return jdata



def writeScene(myScene, verbose):
    # save it to an output file
    outputFile = myScene["jsonFileName"]

    print(os.path.isfile(outputPath + outputFile))

    # if there is a file already there
    print(outputPath + outputFile)
    fileExists = os.path.isfile(outputPath + outputFile)

    if verbose:
        while (fileExists or outputFile == "" ):
    
            print("This is a listing of files in this path: ")
            flist = os.listdir(outputPath)
            for f in flist:
                print(f)
            yn = input("The file " + outputPath + outputFile +" already exists \n Do you want to overwrite it? ")
            if yn == "y" or yn == "Y":
                fileExists = False
            else:
                outputFile = input("What is the new name for our file? ")
                fileExists = os.path.isfile(outputPath + outputFile)
                

    print("\nWriting " + outputPath+outputFile + "\n\n")


    # form the final scene as a string
    myScene["jsonFileName"] = outputFile
    finalJson = json.dumps(myScene, indent=4)
    f = open(outputPath + outputFile, "w", encoding='utf-8')
    f.write(json.dumps(myScene, ensure_ascii=False, indent=4))
    f.close()

    
def createScene(moduleDescription, sceneData, olist, verbose=False):
    sceneData["objects"] = olist
    
    
    # make a copy of the generic scene for the activity file
    myScene = jdata['genericActivity.json'].copy()

    # apply the module description data ot the sceneObject
    myScene.update(moduleDescription)
    myScene.update(sceneData)


    # we can dump the scene to see what it looks like so far  
    if verbose:
        print("This is the Scene\n\n\n")
        print(json.dumps(myScene, indent=4))    
    
    writeScene(myScene, verbose)    
    
    
# this is a code stub for merging two activity files together
def concatenateScenes( outputPath, outputFile, sceneList):

    print(outputPath, outputFile)
    # open the new file
    f = open(outputPath + outputFile, 'w', encoding='utf-8')
    
    for i in range(len(sceneList)):
        print("  \n scene")
        print(i)
        currentScene = sceneList[i]
        print(currentScene)
        
        # read in the files
        fn1 = outputPath + currentScene
        s1 = jsonFromFile(fn1)

        # Dump the file - but do NOT use any indents.  This will flatten
        # the file into a single line.  We then write it out as a line with a newline character.
        a1 = json.dumps(s1, ensure_ascii=False)
        f.write(a1 + "\n")

    # close it
    f.close()
    
def findKeyDiffs(baseClass, olist):
    # find the differences between two dictionaries
    
    # get the keys for the base class
    baseKeys = list(baseClass.keys())
    
    # find the set of common keys for the objects and the set of unique keys
    commonKeys = []
    uniqueKeys = []
    
    # get the key for each object
    for ii in range(len(olist)):
        cc = olist[i]
        objectKeys = list(cc.keys())
    
        # the union of the keys
        uKeys = list(set(baseKeys + objectKeys))
        
        # the intersection of keys
        iKeys = list(set(baseKeys).intersection(objectKeys))
        
        # find the elements only in one list or the other
        aKeys = list( set(baseKeys)^set(objectKeys))

        commonKeys = commonKeys + uKeys
        uniqueKeys = uniqueKeys + aKeys
        
    
    commonKeys = list(set(commonKeys))
    uniqueKeys = list(set(uniqueKeys))
    
    return commonKeys, uniqueKeys
    
        #for ii in range(len(k1)):
        #    k = k1[ii]
        #    if o1[k] != o2[k]:
        #        print(k)
    
    #


def findDiffs(commonKeys, baseClass, olist):
    
    # for each key
    for c in commonKeys:
        baseValue = baseClass[c]
        
        # are there any differences
        diffs = False
        for i in range(len(olist)):
            newValue = olist[i][c]
            if newValue != baseValue:
                diffs = True


        if diffs:
            print("{0:>15}".format(c), end="")
            for i in range(len(olist)):

                if baseValue != olist[i][c]:

                    print("  X   ", end="")
                else:
                    print("  .   ", end="")

            print()
    print(len(olist))
    
#commonKeys, uniqueKeys = findKeyDiffs(earthObject, sortList)   
#findDiffs(commonKeys, earthObject, sortList)


def createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale):
    
    tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }



    position1 = {"x" :topPosition['x'] + lineNumber * offSet['x'],
                "y": topPosition['y'] + lineNumber * offSet['y'],
                "z":topPosition['z'] + lineNumber * offSet['z']
               }

    tdata["textField"] = theText
    textObject = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": position1.copy(),
        "rotation": rotationAngle,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": parentObject,
        "name": textObjectName
    }
    
    return textObject.copy()



def createObjectModification(
    name, # object name
    activationCondition, 
    reactivateObject=False, 
    parentObject="[DYNAMIC]", 
    audioClipName = "",
    goCallback ="",
    newColor= None,
    enable=True, 
    newPosition=None, 
    newEulerAngles=None, 
    newScale=None,
    tmp=None):
    
    
    mm = {"name": name,
         "parentObject": parentObject,
         "activationConditions" : activationCondition,
         "reactiveObject": reactivateObject,
          "enable": enable,
          "position": {"x":0.0, "y":0.0, "z":0.0},
          "newPosition": False,
          "eulerAngles": {"x":0.0, "y":0.0, "z":0.0},
          "newEulerAngles": False,
          "scale": {"x":0.0, "y":0.0, "z":0.0},
          "newScale": False,
             }
                        
    if newPosition != None:
        mm["position"] = newPosition
        mm["newPosition"] = True
            
    if newEulerAngles != None:
        mm["eulerAngles"] = newEulerAngles
        mm["newEulerAngles"] = True
            
    if newScale != None:
        mm["scale"] = newScale
        mm["newScale"] = True
            
    if newColor != None:
        mm['color'] = newColor
    
    if tmp != None:
        mm['tmp'] = tmp
        
    return mm.copy()


def createButton(name, 
                 label, 
                 position, 
                 labelOffset = 0.1,
                 scale = 1, 
                 parentObject = "[_DYNAMIC]",
                 color=[255,0,0,255], 
                 eulerAngles= {"x":90, "y":180.0, "z":0.0}):

    
    scriptDataA = {
    "name": "demoButtonActions",
    "callBackObjects": ["demoModule"]
    }
    scriptDataA = json.dumps(scriptDataA) 


    brbData = {
                "type": "Prefabs/demoButtonPrefab",
                "parentName": parentObject,
                "position": position,
                "eulerAngles": eulerAngles,
                "scale": {"x":scale, "y":scale, "z":scale},
                "name": name, 
                "color": color,
                "componentsToAdd": [scriptDataA]
            }
    
    labelPosition  = {"x":0.0, "y":0.0, "z":labelOffset}
    tdata = {
        "textField": "",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }
    tdata["textField"] = label
    labelScale = 0.6 * scale
    brbLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": labelPosition,
        "eulerAngles": eulerAngles,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": name,
        "name": name+"Label"  
    }
    return brbData, brbLabel




def flattenObjects(current):
    infoDictionary = {}
    columnData = ['name', 'parentName', 'position','rotation','scale','texture', 'childName','childColor','componentsToAdd']
    for c in columnData:
        infoDictionary[c] = []

    for o in current["objects"]:
        for c in columnData:    
            try:
                data = o[c]
            except:
                data = ""
            infoDictionary[c].append(data)
        
    return infoDictionary


# Read in an input file

In [15]:
flist = [  'moonEarthSizes.json',
    'synchronousrotation.json',
    'phasesFromEarth.json',
    'phasesOfEarth.json',
    'northpoleview.json',
    'basketballPhases.json',
    'basketballPhases2.json',
    'finalSummary.json']

choice = 0

path = "./"

jdata = loadFileIntoDictionary(path, flist)



In [ ]:
'''
od = np.empty( (len(current), 10), dtype=np.dtype("U100"))
print(name)
    od[ii,0] = name
    od[ii,1] = parent
    od[ii,2] = otype
    od[ii,3] = otype
    ii = ii + 1

od
'''

In [93]:
fnumber = 1
current = jdata[flist[fnumber]]
#print(current)
moduleName = current['moduleName']
specificName = current['specificName']
infoDictionary = flattenObjects(current)

print(moduleName + " ---  " + specificName)

display =2
if display == 1:
    for k in infoDictionary.keys():
        print("--")
        print(k, infoDictionary[k])
    
    
dframe = pd.DataFrame.from_dict(infoDictionary)

if display == 2:
    print(dframe)

Synchronous Rotation ---  Synchronous Rotation
                      name         parentName  \
0                      Sun                      
1                    Earth                      
2                     Moon                      
3                    Robot               Moon   
4             moonLabelTop               Moon   
5          moonLabelBottom               Moon   
6                taskLabel         [_DYNAMIC]   
7             sectionLabel         [_DYNAMIC]   
8            brbNextModule       sectionLabel   
9       brbNextModuleLabel      brbNextModule   
10       brbPreviousModule       sectionLabel   
11  brbPreviousModuleLabel  brbPreviousModule   
12             brbNextClip          taskLabel   
13        brbNextClipLabel        brbNextClip   
14         brbPreviousClip          taskLabel   
15    brbPreviousClipLabel    brbPreviousClip   

                            position                        rotation  \
0       {'x': -39.0, 'y': 0, 'z': 0}           

In [117]:




def flattenClips(current):
    
    
    clipDictionary = {}
    

    clipInfo = ['clipName', 'audioClipString', 'timeToEnd', 'autoAdvance', "changes"]
    objectMods1 = ['newPosition', 'newEulerAngles', "newScale"]

    objectNotation1 = ["P", "R", "S"]
    
    objectMods2 = ["canvasText", 'tmp']
    objectNotation2 = [ "C", "T"]
    
    for c in clipInfo:
        clipDictionary[c] = []

    # get the basic information
    for o in current["clips"]:
        for c in clipInfo:    
            try:
                data = o[c]
            except:
                data = ""
            clipDictionary[c].append(data)
            
        try:
            objectChanges = o['objectChanges']  
        except: 
            objectChanges = []

        # now summarize the object changes
        for o in objectChanges:
            
            name = o['name']
            s = ""
            for j in range(len(objectMods1)):
                m = objectMods1[j]
                try:
                    om = o[m]
                    if om == True:
                        ss = objectNotation1[j]
                    else:
                        ss = ""
                except:
                    ss = ""
                s = s + ss
                
            for j in range(len(objectMods2)):
                m = objectMods2[j]
                try:
                    om = o[m]
                    ss = objectNotation2[j]
                except:
                    ss = ""
                s = s + ss
            clipDictionary['changes'].append([name,s])
            
                
    
    return clipDictionary


clipDictionary = flattenClips(current)

for c in clipDictionary:
    for k in c.keys():
        print(k, c[k])
    
#dframe2 = pd.DataFrame.from_dict(clipDictionary)
#print(dframe2)
    

AttributeError: 'str' object has no attribute 'keys'

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [60]:
current

'd'

In [66]:
current

{'moduleName': 'Phases on a Basketballl',
 'specificName': 'Phases on a BasketBall',
 'jsonFileName': 'basketballPhases.json',
 'prefabName': 'demoPrefab',
 'prerequisiteActivities': [],
 'educationalObjectives': ['Students will learn that phases are caused by the angle between the Moon, Earth, and Sun'],
 'instructions': ['NA'],
 'numRepeatsAllowed': 0,
 'numGradableRepeatsAllowed': 0,
 'gradingCriteria': '',
 'currentScore': 0.0,
 'bestScore': 0.0,
 'completed': False,
 'currentSubphase': 0,
 'subphaseNames': [],
 'urlJson': '',
 'json': '',
 'timeToEnd': 2000,
 'endUsingButton': True,
 'description': 'A demonstration of how moon phases occur',
 'author': 'John Wallin',
 'authorInstitution': 'Middle Tennessee State University',
 'dateCreated': 'Thu Sep  2 15:17:23 2021',
 'introAudio': '',
 'createObjects': True,
 'destroyObjects': True,
 'restoreLights': True,
 'objects': [{'type': 'Prefabs/SunPrefab',
   'position': {'x': -39.0, 'y': 0, 'z': 0},
   'scale': {'x': 5, 'y': 5, 'z': 5}